**Support code for Exploratory Report** <br>
https://datastudio.google.com/reporting/ce9b69b3-3d9b-4aee-bb62-7baab90a0eca

In [1]:
#!pip install web3

In [2]:
import json
from datetime import datetime
from decimal import Decimal
import pandas as pd
from web3 import Web3
from web3.exceptions import (InvalidEventABI, LogTopicError, MismatchedABI)
from web3._utils.events import get_event_data
from eth_utils import (encode_hex, event_abi_to_log_topic)


# The dafault RPC from ethersjs, change it if it doesn't work: https://infura.io/docs
RPC_Endpoint = 'https://mainnet.infura.io/v3/84842078b09946638c03157f83405213'

w3_eth = Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 20}))
print ('Ethereum connected:', w3_eth.isConnected())


Ethereum connected: True


In [3]:
def get_event_abi(abi, abi_name, abi_type = 'event'):
    l = [x for x in abi if x['type'] == abi_type and x['name']==abi_name]
    return l[0]
def get_events(abi_dict, abi_type = 'event'):
    events = []
    for contract_type in abi_dict.keys():
        for event in [x for x in abi_dict[contract_type] if x['type'] == 'event']:
            events.append({'contract_type': contract_type,
                           'event'        : event['name'],
                           'topic'        : encode_hex(event_abi_to_log_topic(event)),
                           'abi'          : event})
    
    df= pd.DataFrame(events)
    return df
def get_logs (w3, contract_address, topics, from_block=0, to_block=None):

    if not to_block:
        to_block = w3.eth.get_block('latest').number
    
    try:
        logs = w3.eth.get_logs({"address": contract_address
                               ,"topics":topics 
                               ,"fromBlock": from_block 
                               ,"toBlock": to_block
                               })
    except ValueError:
        logs = None
        from_block = 12*10**6
        batch_size = 10**6
        while from_block < to_block:
            print(datetime.utcnow(), f'from_block {from_block} to_block {from_block + batch_size}')
            batch_logs = w3.eth.get_logs({"address": contract_address
                                   ,"topics":topics 
                                   ,"fromBlock": from_block 
                                   ,"toBlock": from_block + batch_size # narrow down the range to avoid too many results error 
                                   })   
            if not logs:
                logs = batch_logs
            else:
                logs = logs + batch_logs
            if len(logs) > 0:    
                print(datetime.utcnow(), f'Total logs count {len(logs)}')  
            
            from_block = from_block + batch_size + 1
            
    all_events = []
    for l in logs:
        try:
            evt_topic0 = l['topics'][0].hex()
            evt_abi = event_topics[event_topics['topic']== evt_topic0]['abi'].values[0] 
            evt = get_event_data(w3.codec, evt_abi, l)
        except MismatchedABI: #if for some reason there are other events 
            pass
        all_events.append(evt)
    df = pd.DataFrame(all_events)
    return df

In [4]:
aggregator_abi_str = """[{"inputs":[{"internalType":"address","name":"_aggregator","type":"address"},{"internalType":"address","name":"_accessController","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"int256","name":"current","type":"int256"},{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"updatedAt","type":"uint256"}],"name":"AnswerUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":true,"internalType":"address","name":"startedBy","type":"address"},{"indexed":false,"internalType":"uint256","name":"startedAt","type":"uint256"}],"name":"NewRound","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"inputs":[],"name":"acceptOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"accessController","outputs":[{"internalType":"contract AccessControllerInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"aggregator","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_aggregator","type":"address"}],"name":"confirmAggregator","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"description","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_roundId","type":"uint256"}],"name":"getAnswer","outputs":[{"internalType":"int256","name":"","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint80","name":"_roundId","type":"uint80"}],"name":"getRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_roundId","type":"uint256"}],"name":"getTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestAnswer","outputs":[{"internalType":"int256","name":"","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRound","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address payable","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint16","name":"","type":"uint16"}],"name":"phaseAggregators","outputs":[{"internalType":"contract AggregatorV2V3Interface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"phaseId","outputs":[{"internalType":"uint16","name":"","type":"uint16"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_aggregator","type":"address"}],"name":"proposeAggregator","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"proposedAggregator","outputs":[{"internalType":"contract AggregatorV2V3Interface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint80","name":"_roundId","type":"uint80"}],"name":"proposedGetRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"proposedLatestRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_accessController","type":"address"}],"name":"setController","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_to","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]
"""
ocr_abi_str = """
[{"inputs":[{"internalType":"uint32","name":"_maximumGasPrice","type":"uint32"},{"internalType":"uint32","name":"_reasonableGasPrice","type":"uint32"},{"internalType":"uint32","name":"_microLinkPerEth","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerObservation","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerTransmission","type":"uint32"},{"internalType":"address","name":"_link","type":"address"},{"internalType":"address","name":"_validator","type":"address"},{"internalType":"int192","name":"_minAnswer","type":"int192"},{"internalType":"int192","name":"_maxAnswer","type":"int192"},{"internalType":"contract AccessControllerInterface","name":"_billingAccessController","type":"address"},{"internalType":"contract AccessControllerInterface","name":"_requesterAccessController","type":"address"},{"internalType":"uint8","name":"_decimals","type":"uint8"},{"internalType":"string","name":"description","type":"string"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"user","type":"address"}],"name":"AddedAccess","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"int256","name":"current","type":"int256"},{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"updatedAt","type":"uint256"}],"name":"AnswerUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"contract AccessControllerInterface","name":"old","type":"address"},{"indexed":false,"internalType":"contract AccessControllerInterface","name":"current","type":"address"}],"name":"BillingAccessControllerSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint32","name":"maximumGasPrice","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"reasonableGasPrice","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"microLinkPerEth","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"linkGweiPerObservation","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"linkGweiPerTransmission","type":"uint32"}],"name":"BillingSet","type":"event"},{"anonymous":false,"inputs":[],"name":"CheckAccessDisabled","type":"event"},{"anonymous":false,"inputs":[],"name":"CheckAccessEnabled","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint32","name":"previousConfigBlockNumber","type":"uint32"},{"indexed":false,"internalType":"uint64","name":"configCount","type":"uint64"},{"indexed":false,"internalType":"address[]","name":"signers","type":"address[]"},{"indexed":false,"internalType":"address[]","name":"transmitters","type":"address[]"},{"indexed":false,"internalType":"uint8","name":"threshold","type":"uint8"},{"indexed":false,"internalType":"uint64","name":"encodedConfigVersion","type":"uint64"},{"indexed":false,"internalType":"bytes","name":"encoded","type":"bytes"}],"name":"ConfigSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":true,"internalType":"address","name":"startedBy","type":"address"},{"indexed":false,"internalType":"uint256","name":"startedAt","type":"uint256"}],"name":"NewRound","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint32","name":"aggregatorRoundId","type":"uint32"},{"indexed":false,"internalType":"int192","name":"answer","type":"int192"},{"indexed":false,"internalType":"address","name":"transmitter","type":"address"},{"indexed":false,"internalType":"int192[]","name":"observations","type":"int192[]"},{"indexed":false,"internalType":"bytes","name":"observers","type":"bytes"},{"indexed":false,"internalType":"bytes32","name":"rawReportContext","type":"bytes32"}],"name":"NewTransmission","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"transmitter","type":"address"},{"indexed":false,"internalType":"address","name":"payee","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"OraclePaid","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"transmitter","type":"address"},{"indexed":true,"internalType":"address","name":"current","type":"address"},{"indexed":true,"internalType":"address","name":"proposed","type":"address"}],"name":"PayeeshipTransferRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"transmitter","type":"address"},{"indexed":true,"internalType":"address","name":"previous","type":"address"},{"indexed":true,"internalType":"address","name":"current","type":"address"}],"name":"PayeeshipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"user","type":"address"}],"name":"RemovedAccess","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"contract AccessControllerInterface","name":"old","type":"address"},{"indexed":false,"internalType":"contract AccessControllerInterface","name":"current","type":"address"}],"name":"RequesterAccessControllerSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"requester","type":"address"},{"indexed":false,"internalType":"bytes16","name":"configDigest","type":"bytes16"},{"indexed":false,"internalType":"uint32","name":"epoch","type":"uint32"},{"indexed":false,"internalType":"uint8","name":"round","type":"uint8"}],"name":"RoundRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previous","type":"address"},{"indexed":true,"internalType":"address","name":"current","type":"address"}],"name":"ValidatorUpdated","type":"event"},{"inputs":[],"name":"LINK","outputs":[{"internalType":"contract LinkTokenInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"acceptOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"}],"name":"acceptPayeeship","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_user","type":"address"}],"name":"addAccess","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"billingAccessController","outputs":[{"internalType":"contract AccessControllerInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"checkEnabled","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"description","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"disableAccessCheck","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"enableAccessCheck","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_roundId","type":"uint256"}],"name":"getAnswer","outputs":[{"internalType":"int256","name":"","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBilling","outputs":[{"internalType":"uint32","name":"maximumGasPrice","type":"uint32"},{"internalType":"uint32","name":"reasonableGasPrice","type":"uint32"},{"internalType":"uint32","name":"microLinkPerEth","type":"uint32"},{"internalType":"uint32","name":"linkGweiPerObservation","type":"uint32"},{"internalType":"uint32","name":"linkGweiPerTransmission","type":"uint32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint80","name":"_roundId","type":"uint80"}],"name":"getRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_roundId","type":"uint256"}],"name":"getTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_user","type":"address"},{"internalType":"bytes","name":"_calldata","type":"bytes"}],"name":"hasAccess","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestAnswer","outputs":[{"internalType":"int256","name":"","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestConfigDetails","outputs":[{"internalType":"uint32","name":"configCount","type":"uint32"},{"internalType":"uint32","name":"blockNumber","type":"uint32"},{"internalType":"bytes16","name":"configDigest","type":"bytes16"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRound","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestTransmissionDetails","outputs":[{"internalType":"bytes16","name":"configDigest","type":"bytes16"},{"internalType":"uint32","name":"epoch","type":"uint32"},{"internalType":"uint8","name":"round","type":"uint8"},{"internalType":"int192","name":"latestAnswer","type":"int192"},{"internalType":"uint64","name":"latestTimestamp","type":"uint64"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"linkAvailableForPayment","outputs":[{"internalType":"int256","name":"availableBalance","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"maxAnswer","outputs":[{"internalType":"int192","name":"","type":"int192"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"minAnswer","outputs":[{"internalType":"int192","name":"","type":"int192"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_signerOrTransmitter","type":"address"}],"name":"oracleObservationCount","outputs":[{"internalType":"uint16","name":"","type":"uint16"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"}],"name":"owedPayment","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address payable","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_user","type":"address"}],"name":"removeAccess","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"requestNewRound","outputs":[{"internalType":"uint80","name":"","type":"uint80"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"requesterAccessController","outputs":[{"internalType":"contract AccessControllerInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint32","name":"_maximumGasPrice","type":"uint32"},{"internalType":"uint32","name":"_reasonableGasPrice","type":"uint32"},{"internalType":"uint32","name":"_microLinkPerEth","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerObservation","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerTransmission","type":"uint32"}],"name":"setBilling","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract AccessControllerInterface","name":"_billingAccessController","type":"address"}],"name":"setBillingAccessController","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"_signers","type":"address[]"},{"internalType":"address[]","name":"_transmitters","type":"address[]"},{"internalType":"uint8","name":"_threshold","type":"uint8"},{"internalType":"uint64","name":"_encodedConfigVersion","type":"uint64"},{"internalType":"bytes","name":"_encoded","type":"bytes"}],"name":"setConfig","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"_transmitters","type":"address[]"},{"internalType":"address[]","name":"_payees","type":"address[]"}],"name":"setPayees","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract AccessControllerInterface","name":"_requesterAccessController","type":"address"}],"name":"setRequesterAccessController","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_newValidator","type":"address"}],"name":"setValidator","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_to","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"},{"internalType":"address","name":"_proposed","type":"address"}],"name":"transferPayeeship","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes","name":"_report","type":"bytes"},{"internalType":"bytes32[]","name":"_rs","type":"bytes32[]"},{"internalType":"bytes32[]","name":"_ss","type":"bytes32[]"},{"internalType":"bytes32","name":"_rawVs","type":"bytes32"}],"name":"transmit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"transmitters","outputs":[{"internalType":"address[]","name":"","type":"address[]"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"validator","outputs":[{"internalType":"contract AggregatorValidatorInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_recipient","type":"address"},{"internalType":"uint256","name":"_amount","type":"uint256"}],"name":"withdrawFunds","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"}],"name":"withdrawPayment","outputs":[],"stateMutability":"nonpayable","type":"function"}]
"""
aggregator_abi = json.loads(aggregator_abi_str)
ocr_abi = json.loads(ocr_abi_str)
event_topics = get_events({'Aggregator': aggregator_abi, 'OCR': ocr_abi})

pricefeed_contracts = {
     'eth-usd' : [w3_eth, Web3.toChecksumAddress('0x37bC7498f4FF12C19678ee8fE19d713b87F6a9e6'), 0.005, 'usd', 3600]
    
    ,'btc-eth' : [w3_eth, Web3.toChecksumAddress('0x81076d6ff2620ea9dd7ba9c1015f0d09a3a732e6'), 0.02, 'eth', 86400]
    ,'btc-usd' : [w3_eth, Web3.toChecksumAddress('0xAe74faA92cB67A95ebCAB07358bC222e33A34dA7'), 0.005, 'usd', 3600]
    
    ,'usdc-eth': [w3_eth, Web3.toChecksumAddress('0xe5bbbdb2bb953371841318e1edfbf727447cef2e'), 0.01, 'eth', 86400]
    ,'usdc-usd': [w3_eth, Web3.toChecksumAddress('0x789190466E21a8b78b8027866CBBDc151542A26C'), 0.0025, 'usd', 86400]
    
    ,'dai-eth' : [w3_eth, Web3.toChecksumAddress('0x158228e08c52f3e2211ccbc8ec275fa93f6033fc'), 0.01, 'eth', 86400]
    ,'dai-usd' : [w3_eth, Web3.toChecksumAddress('0xDEc0a100eaD1fAa37407f0Edc76033426CF90b82'), 0.0025, 'usd', 3600]
    
    ,'1inch-eth': [w3_eth, Web3.toChecksumAddress('0xb2f68c82479928669b0487d1daed6ef47b63411e'), 0.02, 'eth', 86400]
    ,'1inch-usd': [w3_eth, Web3.toChecksumAddress('0xd2bdD1E01fd2F8d7d42b209c111c7b32158b5a42'), 0.01, 'usd', 86400]
    
    ,'aave-eth': [w3_eth, Web3.toChecksumAddress('0xdf0da6b3d19e4427852f2112d0a963d8a158e9c7'), 0.02, 'eth', 86400]
    ,'aave-usd': [w3_eth, Web3.toChecksumAddress('0xe3f0dEdE4B499c07e12475087AB1A084b5F93bc0'), 0.01, 'usd', 3600]
    
    ,'comp-eth': [w3_eth, Web3.toChecksumAddress('0x9d6acd34d481512586844fd65328bd358d306752'), 0.02, 'eth', 86400]     
    ,'comp-usd': [w3_eth, Web3.toChecksumAddress('0x6eaC850f531d0588c0114f1E93F843B78669E6d2'), 0.01, 'usd', 3600]     
    
    ,'dpi-eth': [w3_eth, Web3.toChecksumAddress('0x36e4f71440edf512eb410231e75b9281d4fcfc4c'), 0.02, 'eth', 86400]     
    ,'dpi-usd': [w3_eth, Web3.toChecksumAddress('0x68f1b8317c19ff02fb68a8476c1d3f9fc5139c0a'), 0.01, 'usd', 3600]     

    ,'fei-eth': [w3_eth, Web3.toChecksumAddress('0x4be991b4d560bba8308110ed1e0d7f8da60acf6a'), 0.02, 'eth', 86400] 
    ,'fei-usd': [w3_eth, Web3.toChecksumAddress('0x1D244648d5a63618751d006886268Ae3550d0Dfd'), 0.01, 'usd', 3600] 
    
    ,'link-eth': [w3_eth, Web3.toChecksumAddress('0xbba12740de905707251525477bad74985dec46d2'), 0.01, 'eth', 21600] 
    ,'link-usd': [w3_eth, Web3.toChecksumAddress('0xDfd03BfC3465107Ce570a0397b247F546a42D0fA'), 0.01, 'usd', 3600] 

    ,'snx-eth': [w3_eth, Web3.toChecksumAddress('0xbafe3cb0e563e914806a99d547bdbf2cfcf5fdf6'), 0.02, 'eth', 86400] 
    ,'snx-usd': [w3_eth, Web3.toChecksumAddress('0x06ce8Be8729B6bA18dD3416E3C223a5d4DB5e755'), 0.01, 'usd', 86400] 

    ,'uni-eth': [w3_eth, Web3.toChecksumAddress('0xc1d1d0da0fcf78157ea25d0e64e3be679813a1f7'), 0.02, 'eth', 86400] 
    ,'uni-usd': [w3_eth, Web3.toChecksumAddress('0x68577f915131087199Fe48913d8b416b3984fd38'), 0.01, 'usd', 3600] 

    ,'yfi-eth': [w3_eth, Web3.toChecksumAddress('0xaa5aa80e416f9d32ffe6c390e24410d02d203f70'), 0.01, 'eth', 86400]
    ,'yfi-usd': [w3_eth, Web3.toChecksumAddress('0x8a4D74003870064d41D4f84940550911FBfCcF04'), 0.01, 'usd', 3600]
    
    ,'luna-eth': [w3_eth, Web3.toChecksumAddress('0x1a8ac67a1b64f7fd71bb91c21581f036abe6aec2'), 0.02, 'eth', 86400]
    
    ,'ftm-eth': [w3_eth, Web3.toChecksumAddress('0xbdb80d19dea36eb7f63bdfd2bdd4033b2b7e8e4d'), 0.03, 'eth', 86400]
     
    ,'sushi-eth': [w3_eth, Web3.toChecksumAddress('0xd01bbb3afed2cb5ca92ca3834d441dc737f0da70'), 0.02, 'eth', 86400]
    ,'sushi-usd': [w3_eth, Web3.toChecksumAddress('0x7213536a36094cD8a768a5E45203Ec286Cba2d74'), 0.01, 'usd', 3600]
    
    ,'crv-eth': [w3_eth, Web3.toChecksumAddress('0x7f67ca2ce5299a67acd83d52a064c5b8e41ddb80'), 0.02, 'eth', 86400]
    ,'crv-usd': [w3_eth, Web3.toChecksumAddress('0xb4c4a493AB6356497713A78FFA6c60FB53517c63'), 0.01, 'usd', 86400]
    
    ,'cvx-usd': [w3_eth, Web3.toChecksumAddress('0x8d73ac44bf11cadcdc050bb2bccae8c519555f1a'), 0.02, 'usd', 86400] 
    
    ,'ldo-eth': [w3_eth, Web3.toChecksumAddress('0x7898AcCC83587C3C55116c5230C17a6Cd9C71bad'), 0.02, 'eth', 86400] 
    
    ,'steth-eth': [w3_eth, Web3.toChecksumAddress('0x716BB759A5f6faCdfF91F0AfB613133d510e1573'), 0.02, 'eth', 86400] 
    ,'steth-usd': [w3_eth, Web3.toChecksumAddress('0xdA31bc2B08F22AE24aeD5F6EB1E71E96867BA196'), 0.01, 'usd', 3600] 
    
    ,'frax-eth': [w3_eth, Web3.toChecksumAddress('0x56f98706C14DF5C290b02Cec491bB4c20834Bb51'), 0.02, 'eth', 86400] 
    ,'frax-usd': [w3_eth, Web3.toChecksumAddress('0x61eB091ea16A32ea5B880d0b3D09d518c340D750'), 0.01, 'usd', 3600] 

    ,'lusd-usd': [w3_eth, Web3.toChecksumAddress('0x27b97a63091d185cE056e1747624b9B92BAAD056'), 0.01, 'usd', 3600] 

    ,'susd-eth': [w3_eth, Web3.toChecksumAddress('0x45bb69B89D60878d1e42522342fFCa9F2077dD84'), 0.01, 'eth', 86400] 
    ,'susd-usd': [w3_eth, Web3.toChecksumAddress('0x1187272A0E3A603eC4734CeC73a0880055eCC593'), 0.005, 'usd', 86400] 

    ,'gusd-eth': [w3_eth, Web3.toChecksumAddress('0x9c2C487DAd6C8e5bb49dC6908a29D95a234FaAd8'), 0.02, 'eth', 86400] 
    ,'gusd-usd': [w3_eth, Web3.toChecksumAddress('0x6a805f2580b8D75d40331c26C074c2c42961E7F2'), 0.005, 'usd', 86400] 

    ,'usdt-eth': [w3_eth, Web3.toChecksumAddress('0x7De0d6fce0C128395C488cb4Df667cdbfb35d7DE'), 0.01, 'eth', 86400] 
    ,'usdt-usd': [w3_eth, Web3.toChecksumAddress('0xa964273552C1dBa201f5f000215F5BD5576e8f93'), 0.0025, 'usd', 86400] 
    }

config = {'Ethereum':pricefeed_contracts, 
         }

In [5]:
#from google.oauth2 import service_account
#import pandas_gbq

#try:
#    df_blocknum = pandas_gbq.read_gbq('select max(blockNumber) from gearbox.oracle_price_history', 
#                             project_id=gcp_project_id,
#                             progress_bar_type = None,)
#    from_block = int(df_blocknum.iloc[0,0]) + 1
#except pandas_gbq.exceptions.GenericGBQException:
#    print('The table does not exist?')
#    from_block = 0
from_block = 0
print(datetime.utcnow(),'from_block=', from_block)

df = pd.DataFrame()

topic_AnswerUpdated = event_topics[(event_topics['event']=='AnswerUpdated') & (event_topics['contract_type']=='OCR')]['topic'].values[0]

for chain in config.keys():
    chain_contracts = config[chain]
    for ticker in chain_contracts.keys():
        print(f'{datetime.utcnow()} : {ticker.upper()}')
        w3 = chain_contracts[ticker][0]
        address = chain_contracts[ticker][1]
        threshold = chain_contracts[ticker][2]
        base = chain_contracts[ticker][3]
        heartbeat = chain_contracts[ticker][4]
        
        decimals = w3.eth.contract(address=address, abi=aggregator_abi_str).functions.decimals().call()

        df_ticker = get_logs(w3, address, [topic_AnswerUpdated], from_block)
        df_ticker[['chain', 'ticker', 'threshold', 'base', 'heartbeat', 'decimals', 'type']] = [chain , ticker, threshold, base, heartbeat, decimals, 'direct']
        
        df_ticker['tx_hash']       = df_ticker['transactionHash'].apply(lambda x: x.hex())        
        df_ticker['updated_at']    = df_ticker['args'].apply(lambda x: pd.to_datetime(x['updatedAt'], unit='s'))
        df_ticker['price']         = df_ticker['args'].apply(lambda x: x['current'])
        df_ticker['price_decimal'] = df_ticker['price']/10**(decimals)
        df_ticker['deviation']     = df_ticker['price']/df_ticker['price'].shift(1) - 1
        
        df = df.append(df_ticker[['ticker', 'updated_at', 'chain','price', 'price_decimal', 'type', 'deviation', 'threshold', 'base', 'heartbeat', 'decimals', 'blockNumber','tx_hash']], ignore_index=True)
        print(f'''{datetime.utcnow()} : {ticker.upper()}: observation count = {len(df_ticker)}, price range: {df_ticker['price_decimal'].min()}-{df_ticker['price_decimal'].max()}''')
        
        if ticker == 'eth-usd':
            df_eth_usd = df_ticker
        elif base == 'usd': # usd2eth cross rate
            ticker = ticker.replace('-usd','-eth')
            if ticker not in chain_contracts.keys():
                print(f'{datetime.utcnow()}: cross rate {ticker.upper()}')
                df_cross = pd.DataFrame()
                for k, t in df_ticker.iterrows(): 
                    e = df_eth_usd[df_eth_usd['updated_at'] <= t['updated_at']]
                    if len(e) > 0:
                        i = (t['updated_at'] - e['updated_at']).idxmin() 
                        e_price_decimal, e_decimals = e.loc[i][['price_decimal','decimals']]
                        t.ticker = ticker
                        t.price_decimal = t.price_decimal/e_price_decimal
                        t.decimals = 18
                        t.type = 'cross'
                        t.price = int(t.price_decimal*10**(t.decimals))
                        df_cross = df_cross.append(t)
                df_cross['deviation']     = df_cross['price']/df_cross['price'].shift(1) - 1
                df = df.append(df_cross[['ticker', 'updated_at', 'chain','price', 'price_decimal', 'type', 'deviation', 'threshold', 'base', 'heartbeat', 'decimals', 'blockNumber','tx_hash']], ignore_index=True)
                print(f'''{datetime.utcnow()}: cross rate {ticker.upper()}: observation count = {len(df_cross)}, price range: {df_cross['price_decimal'].min()}-{df_cross['price_decimal'].max()}''')
        elif base == 'eth': #eth2usd cross rate
            ticker = ticker.replace('-eth','-usd')
            if ticker not in chain_contracts.keys():
                print(f'{datetime.utcnow()}: cross rate {ticker.upper()}')
                df_cross = pd.DataFrame()
                for k, t in df_ticker.iterrows(): 
                    e = df_eth_usd[df_eth_usd['updated_at'] <= t['updated_at']]
                    if len(e) > 0:
                        i = (t['updated_at'] - e['updated_at']).idxmin() 
                        e_price_decimal, e_decimals = e.loc[i][['price_decimal','decimals']]
                        t.ticker = ticker
                        t.decimals = e_decimals
                        t.type = 'cross'
                        t.price_decimal = t.price_decimal*e_price_decimal
                        t.price = int(t.price_decimal*10**(t.decimals))
                        df_cross = df_cross.append(t)
                df_cross['deviation']     = df_cross['price']/df_cross['price'].shift(1) - 1
                df = df.append(df_cross[['ticker', 'updated_at', 'chain','price', 'price_decimal', 'type', 'deviation', 'threshold', 'base', 'heartbeat', 'blockNumber','tx_hash']], ignore_index=True)
                print(f'''{datetime.utcnow()}: cross rate {ticker.upper()}: observation count = {len(df_cross)}, price range: {df_cross['price_decimal'].min()}-{df_cross['price_decimal'].max()}''')
                
df = df.sort_values(by = 'updated_at').reset_index(drop=True)
pd.set_option("precision", 18)

2022-04-15 08:56:48.121741 from_block= 0
2022-04-15 08:56:48.125304 : ETH-USD
2022-04-15 08:56:53.154209 from_block 12000000 to_block 13000000
2022-04-15 08:57:04.191202 Total logs count 8469
2022-04-15 08:57:04.191331 from_block 13000001 to_block 14000001
2022-04-15 08:57:14.037620 Total logs count 17305
2022-04-15 08:57:14.037769 from_block 14000002 to_block 15000002
2022-04-15 08:57:19.279086 Total logs count 22463
2022-04-15 08:57:34.813601 : ETH-USD: observation count = 22463, price range: 1720.62935399-4850.92
2022-04-15 08:57:34.814015 : BTC-ETH
2022-04-15 08:57:36.945100 : BTC-ETH: observation count = 769, price range: 11.45671593-33.65841927889143
2022-04-15 08:57:36.945634 : BTC-USD
2022-04-15 08:57:41.547937 from_block 12000000 to_block 13000000
2022-04-15 08:57:43.744258 Total logs count 6451
2022-04-15 08:57:43.744543 from_block 13000001 to_block 14000001
2022-04-15 08:57:52.828402 Total logs count 13807
2022-04-15 08:57:52.828635 from_block 14000002 to_block 15000002
2022

2022-04-15 09:03:48.480417 : LDO-ETH: observation count = 4216, price range: 0.000473154043167557-0.00225298
2022-04-15 09:03:48.480831: cross rate LDO-USD
2022-04-15 09:04:27.880215: cross rate LDO-USD: observation count = 4093, price range: 1.196048253399814-7.238858687204216
2022-04-15 09:04:27.880753 : STETH-ETH
2022-04-15 09:04:28.941363 : STETH-ETH: observation count = 236, price range: 0.98081098-1.0000460908852682
2022-04-15 09:04:28.941956 : STETH-USD
2022-04-15 09:04:40.748381 : STETH-USD: observation count = 6480, price range: 2187.0133105-4813.89441143
2022-04-15 09:04:40.748926 : FRAX-ETH
2022-04-15 09:04:43.747290 : FRAX-ETH: observation count = 1329, price range: 0.000208361327495522-0.00060311328
2022-04-15 09:04:43.747729 : FRAX-USD
2022-04-15 09:04:46.757165 : FRAX-USD: observation count = 1641, price range: 0.987081-1.024
2022-04-15 09:04:46.757757 : LUSD-USD
2022-04-15 09:04:50.084728 : LUSD-USD: observation count = 1650, price range: 0.96821617-1.06929542
2022-04-1

In [6]:
df

,ticker,updated_at,chain,price,price_decimal,type,deviation,threshold,base,heartbeat,decimals,blockNumber,tx_hash
0,comp-eth,2021-02-18 09:09:54,Ethereum,244378668045116040,0.244378668045116043,direct,NaN,0.020000000000000000,eth,86400.0,18.0,11879906.0,0x9610367f2cd00cba48361195cb211f54e5775cf54efc...
1,crv-eth,2021-02-18 10:43:05,Ethereum,1486000000000000,0.001486,direct,NaN,0.020000000000000000,eth,86400.0,18.0,11880352.0,0xa102d125a730c7186a458749e3a8de6323713567c6fa...
2,snx-eth,2021-02-18 10:55:03,Ethereum,12479365000000000,0.012479364999999999,direct,NaN,0.020000000000000000,eth,86400.0,18.0,11880406.0,0x7537d4fce2d92483de4fd133947ec2cde6883f839685...
3,crv-eth,2021-02-18 11:47:08,Ethereum,1454500000000000,0.0014545,direct,-0.021197846567967749,0.020000000000000000,eth,86400.0,18.0,11880656.0,0xb170655b26a54ea69deb98102105e6b11a686338dcb1...
4,ftm-eth,2021-02-18 12:23:53,Ethereum,108390000000000,0.00010839,direct,NaN,0.029999999999999999,eth,86400.0,18.0,11880835.0,0x3a973394035ccfd333f19fbeb09003373e22a269331e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240368,yfi-usd,2022-04-15 08:54:09,Ethereum,1911527857086,19115.278570859998581,direct,0.001452849044063509,0.010000000000000000,usd,3600.0,8.0,14588970.0,0x6611acf4c84d946a0790492b341ce9e780cf38ccf179...
240369,eth-usd,2022-04-15 08:55:43,Ethereum,302353399089,3023.533990890000040963,direct,0.001135117730879598,0.005000000000000000,usd,3600.0,8.0,14588977.0,0x877c8623b2c5b1165da3026b0ef53f67b4672511fc9e...
240370,btc-usd,2022-04-15 08:55:57,Ethereum,4013500000000,40135.0,direct,-0.002819237600833024,0.005000000000000000,usd,3600.0,8.0,14588978.0,0x465744756ee0d01c91bcf1cfb324dd2eb9b6c70e848f...
240371,sushi-usd,2022-04-15 08:55:57,Ethereum,328940458,3.2894045799999998,direct,-0.010109966897381861,0.010000000000000000,usd,3600.0,8.0,14588978.0,0xad735f3bb413398a203bd996fd2bc10c2baf39388f3d...


**Write data frame to GCP BigQuery**

In [7]:
from google.oauth2 import service_account
import pandas_gbq

credentials = service_account.Credentials.from_service_account_file(
    'gearbox-336415-5ed144668529.json',
)
gcp_project_id = 'gearbox-336415'
if len(df)>0:
    numeric_cols = [x for x in df.columns if x in ['price', 'price_decimal' ,'deviation']]
    df[numeric_cols] = df[numeric_cols].astype('float64')

    pandas_gbq.to_gbq(df, 
                      'gearbox.oracle_price_history',
                      project_id=gcp_project_id,
                      if_exists = ('replace' if from_block==0 else 'append'), 
                      progress_bar = False)
    print(datetime.utcnow(), 'gearbox.oracle_price_history, insert done')
else:
    print('No new events since block', from_block)

2022-04-15 09:06:33.250592 gearbox.oracle_price_history, insert done


Exploratory report:

https://datastudio.google.com/reporting/ce9b69b3-3d9b-4aee-bb62-7baab90a0eca